Fitting a WL mass using `clmm`

_the LSST-DESC CLMM team_

In [ ]:
from astropy.cosmology import FlatLambdaCDM
import clmm
from clmm import modeling as mod
from clmm import polaraveraging as avg
import sys
sys.path.append('./support')
import mock_data as mock
from numpy import random
from scipy import optimize as spo

In [ ]:
# define a true cosmology

mock_cosmo = FlatLambdaCDM(H0=70, Om0=0.27, Ob0=0.045)
cosmo = mod.cclify_astropy_cosmo(mock_cosmo)

In [ ]:
needed_type = type(cosmo)
print(needed_type)

# Make mock data

In [ ]:
# define toy cluster parameters

config = {}
config['cosmo'] = cosmo
config['cluster_id'] = "Awesome_cluster"
config['cluster_m'] = 1.e15
config['cluster_z'] = 0.3
config['src_z'] = 0.8
config['concentration'] = 4
config['ngals'] = 10000
config['Delta'] = 200

In [ ]:
# make mock data object

ideal_data = mock.MockData(config=config)

In [ ]:
# populate catalog of galaxy cluster

ideal_data.generate()

In [ ]:
# make a clmm.GalaxyCluster object

cluster_ra = 0.0
cluster_dec = 0.0
gc_object = clmm.GalaxyCluster(config['cluster_id'], cluster_ra, cluster_dec, 
                               config['cluster_z'], ideal_data.catalog)

In [ ]:
# save the clmm.GalaxyCluster object

gc_object.save('mock_GC.pkl')

# Derive observables

In [ ]:
# load a clmm.GalaxyCluster object

cl = clmm.load_cluster('mock_GC.pkl')

ra_l = cl.ra
dec_l = cl.dec
z = cl.z
e1 = cl.galcat['e1']
e2 = cl.galcat['e2']
ra_s = cl.galcat['ra']
dec_s = cl.galcat['dec']

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.scatter(ra_s, dec_s)

In [ ]:
# calculate distance, tangential shear, cross shear for each source galaxy in the cluster

#(theta, g_t, g_x) = avg.compute_shear(ra_l, dec_l, ra_s, dec_s, e1, e2, sky='flat')
(theta, g_t, g_x) = avg.compute_shear(cl)

In [ ]:
type(theta)

In [ ]:
plt.scatter(theta, g_t)


In [ ]:
# define bins

# bins = avg.make_bins(0.1, 3.7, 20)

In [ ]:
rMpc = clmm.polaraveraging._theta_units_conversion(theta,"Mpc",z,mock_cosmo,cosmo_object_type='astropy')

In [ ]:
# calculate shear profiles

(r, gt_profile, gterr_profile) = avg._compute_radial_averages(rMpc, g_t.data)
(r, gx_proflie, gxerr_profile) = avg._compute_radial_averages(rMpc, g_x.data)

In [ ]:
plt.scatter(r, gt_profile)

# Construct model

In [ ]:
# select density profile parametrization and parameter values
def nfw_to_shear_profile(logm):
    m = 10.**logm
    gt_model = mod.predict_reduced_tangential_shear(r, m, config['concentration'], config['cluster_z'], 
                                                    config['src_z'], config['cosmo'], 
                                        Delta=200, halo_profile_parameterization='nfw')
    return (gt_model - gt_profile) **2

# Fit for mass

In [ ]:
# optimize to find the best-fit mass

logm_0 = random.uniform(13., 17., 1)[0]
logm_est = spo.minimize(nfw_to_shear_profile, logm_0)
m_est = 10.**logm_est